In [53]:
import cv2   #include opencv library functions in python
import numpy as np
from pathlib import Path
from tqdm import tqdm

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
checkerboard_size=(8,6)
show_example=False


pathName = Path('D:\StereoCams\Test\Calibration\Left')
globbed = pathName.glob('*.png')
imageList = [x for x in globbed if x.is_file()]

coordinates = np.zeros((len(imageList),np.prod(checkerboard_size),1,2),dtype=np.float32)
# coordinates=[]
objpoints = np.zeros((len(imageList),np.prod(checkerboard_size),3),dtype=np.float32) # 3d point in real world space

pbar = tqdm(     (file for file in imageList)  ,total=len(imageList)   )
for i, fname in enumerate(pbar):
    pbar.set_description("Locating checkerboard coordinates in {}".format(fname.name)) 

    img = cv2.imread(str(fname),0)
    ret, corners = cv2.findChessboardCorners(img,checkerboard_size, flags=cv2.CALIB_CB_ADAPTIVE_THRESH)

    if ret:
        corners2=cv2.cornerSubPix(img,corners, (11,11), (-1,-1), criteria)
        coordinates[i,:,:,:]=corners2[:,:,:]
        objpoints[i,:,:]=objp
        # objpoints.append(objp)
    else:
        print('No checkerboard found in {}, exiting...'.format(fname.name))
        exit()
    
frameDims = img.shape
if show_example==True:
    cv2.drawChessboardCorners(img, (9, 6),corners2, ret)
    cv2.imshow('img', img)

Locating checkerboard coordinates in calibration_frame_11.png: 100%|██████████| 12/12 [00:00<00:00, 69.10it/s]


In [6]:
import numpy as np

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*6,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:8].T.reshape(-1,2)

In [55]:
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints,coordinates,frameDims,None,None)

In [23]:
import numpy as np
import cv2
from pathlib import Path
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
pathName = Path('D:\StereoCams\Test\Calibration\Left')
globbed = pathName.glob('*.png')
for fname in globbed:
    img = cv2.imread(str(fname))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

In [56]:
# temp = list(coordinates)
print(tvecs[0].shape,temp[0].shape)

(3, 1) (48, 1, 2)


In [50]:
coordinates=np.ndarray(coordinates)

TypeError: only integer scalar arrays can be converted to a scalar index

In [34]:
np.save('points.npy',imgpoints,allow_pickle=True)

In [37]:
temp = np.load('points.npy',allow_pickle=True)